In [1]:
import json
import numpy as np
import pandas as pd
import pickle
import copy

In [43]:
rdf = pd.read_csv('./ml-100k/u.data', sep="\t", header=None)
rdf.columns = ["user_id", "item_id", "rating", "timestamp"]
rdf.drop(columns=['timestamp'], inplace=True)
# rdf = rdf.rename(columns={"userId": "user_id", "movieId": "item_id"})
rdf.head()

,user_id,item_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [3]:
item_df = pd.read_csv('./ml-100k/u.item', sep="|",encoding='latin-1',  names=['item_id', 'title','release date','video release date', 'IMDb URL', 'unknown', 'Action', 
                 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 
                 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'])
# item_df = item_df.rename(columns={"movieId": "item_id"})
item_df.head()

,item_id,title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [4]:
user_df = pd.read_csv('./ml-100k/u.user', sep='|', names=["user_id", "age", "gender", "occupation", "zip-code"])
# item_df = item_df.rename(columns={"movieId": "item_id"})
user_df.head()

,user_id,age,gender,occupation,zip-code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [5]:
# occupation_mapping = {0:"other", 1:"academic/educator", 2:"artist", 3:"clerical/admin",
#        4:"college/gradStudent", 5:"customerService", 6:"doctor/healthCare", 7:"executive/managerial", 8:"farmer",
#        9:"homemaker", 10:"K-12Student", 11:"lawyer", 12:"programmer",
#        13:"retired", 14:"sales/marketing", 15:"scientist", 16:"self-employed", 17:"technician/engineer",
#        18:"tradesman/craftsman", 19:"unemployed", 20:"writer"
#              }
# user_df['occupation_text'] = user_df['occupation'].map(occupation_mapping)
user_df.head()

,user_id,age,gender,occupation,zip-code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [6]:
# item_genre_dict = dict()
# for i in range(len(item_df)):
#     genre_str = item_df.at[i, 'genres']
#     genre_list = genre_str.split('|')
#     item_genre_dict[item_df.at[i, 'item_id']] = genre_list

In [7]:
user_type_dict = dict()
for u in range(len(user_df)):
    type_str = user_df.at[u, 'occupation']
    #type_list = type_str.split('|')
    type_list = type_str.split(' ')
    user_type_dict[user_df.at[u, 'user_id']] = type_list

In [8]:
item_set = set(rdf['item_id'].unique())
user_set = set(rdf['user_id'].unique())
print('item num = ' + str(len(item_set)))
print('user num = ' + str(len(user_set)))

item num = 1682
user num = 943


In [9]:
# # count the number for each genre and sort
# import operator
# genre_count = dict()
# for l in item_genre_dict:
#     for g in item_genre_dict[l]:
#         if not g in genre_count:
#             genre_count[g] = 1
#         else:
#             genre_count[g] += 1

# genre_count_sorted = sorted(genre_count.items(), key=operator.itemgetter(1), reverse=True)
# genre_count_sorted

In [10]:
# count the number for each user type and sort
import operator
type_count = dict()
for l in user_type_dict:
    for g in user_type_dict[l]:
        if not g in type_count:
            type_count[g] = 1
        else:
            type_count[g] += 1

type_count_sorted = sorted(type_count.items(), key=operator.itemgetter(1), reverse=True)
type_count_sorted

[('student', 196),
 ('other', 105),
 ('educator', 95),
 ('administrator', 79),
 ('engineer', 67),
 ('programmer', 66),
 ('librarian', 51),
 ('writer', 45),
 ('executive', 32),
 ('scientist', 31),
 ('artist', 28),
 ('technician', 27),
 ('marketing', 26),
 ('entertainment', 18),
 ('healthcare', 16),
 ('retired', 14),
 ('lawyer', 12),
 ('salesman', 12),
 ('none', 9),
 ('homemaker', 7),
 ('doctor', 7)]

In [11]:
key_type = ['engineer','scientist','programmer','educator','executive','artist']

# get the key_type->user_list dict
key_type_user = dict()
for k in key_type:
    key_type_user[k] = list()
for user in user_type_dict:
    for t in user_type_dict[user]:
        if t in key_type:
            key_type_user[t].append(user)

In [12]:
# # collect all the items with key genres
# key_item_set = set()
# for genre in key_genre_item:
#     key_item_set |= set(key_genre_item[genre])

# nonkey_item_set = item_set - key_item_set

In [13]:
# collect all the users with key types
key_user_set = set()
for types in key_type_user:
    key_user_set |= set(key_type_user[types])

nonkey_user_set = user_set - key_user_set

In [14]:
key_user_set

{6,
 13,
 14,
 15,
 17,
 23,
 24,
 25,
 26,
 29,
 31,
 40,
 41,
 45,
 51,
 53,
 54,
 55,
 58,
 59,
 61,
 64,
 65,
 69,
 70,
 71,
 74,
 82,
 84,
 85,
 90,
 93,
 96,
 97,
 98,
 100,
 102,
 105,
 107,
 108,
 111,
 113,
 114,
 115,
 119,
 123,
 133,
 134,
 137,
 141,
 144,
 146,
 148,
 150,
 152,
 156,
 157,
 158,
 160,
 162,
 166,
 171,
 175,
 176,
 177,
 181,
 182,
 183,
 186,
 187,
 189,
 192,
 195,
 200,
 202,
 208,
 209,
 210,
 212,
 213,
 215,
 216,
 219,
 222,
 224,
 227,
 232,
 233,
 235,
 239,
 240,
 242,
 243,
 247,
 250,
 252,
 254,
 260,
 263,
 265,
 267,
 268,
 271,
 272,
 275,
 279,
 283,
 284,
 285,
 290,
 292,
 295,
 297,
 298,
 300,
 302,
 305,
 309,
 310,
 315,
 319,
 321,
 329,
 330,
 335,
 337,
 340,
 343,
 351,
 352,
 353,
 357,
 358,
 364,
 371,
 374,
 379,
 380,
 381,
 382,
 384,
 396,
 402,
 404,
 406,
 407,
 410,
 411,
 412,
 413,
 414,
 415,
 420,
 421,
 426,
 430,
 433,
 435,
 446,
 450,
 467,
 468,
 469,
 470,
 474,
 475,
 479,
 483,
 485,
 486,
 487,
 490,
 492

In [15]:
len(rdf['item_id'])

100000

In [16]:
# remove the non-key type users in rdf
remove_list = []
for user in nonkey_user_set:
    remove_list += rdf.index[rdf['user_id'] == user].values.tolist()

In [17]:
rdf.drop(remove_list, inplace=True)

In [18]:
rdf.reset_index(drop=True, inplace=True)
rating_df = copy.copy(rdf)

In [19]:
rdf = copy.copy(rating_df)

In [20]:
# iteratively remove items and users with less than 2 reviews
rdf.reset_index(drop=True, inplace=True)

rdf['user_freq'] = rdf.groupby('user_id')['user_id'].transform('count')
rdf.drop(rdf.index[rdf['user_freq'] <= 2], inplace=True)
rdf.reset_index(drop=True, inplace=True)
rdf['item_freq'] = rdf.groupby('item_id')['item_id'].transform('count')
rdf.drop(rdf.index[rdf['item_freq'] <= 2], inplace=True)
rdf.reset_index(drop=True, inplace=True)
rdf['user_freq'] = rdf.groupby('user_id')['user_id'].transform('count')
rdf.reset_index(drop=True, inplace=True)
rdf['user_id'].value_counts()

13     590
450    530
537    484
279    393
682    392
      ... 
571     20
572     20
636     20
740     20
242     19
Name: user_id, Length: 319, dtype: int64

In [21]:
item_list = rdf['item_id'].unique()
user_list = rdf['user_id'].unique()
print('item num = ' + str(len(item_list)))
print('user num = ' + str(len(user_list)))
len(rdf['item_id'])

item num = 1177
user num = 319


32704

In [22]:
# get the user and item str id->int id dict
i = 0
user_id_dict = dict()
for u in user_list:
    if not u in user_id_dict:
        user_id_dict[u] = i
        i += 1
j = 0
item_id_dict = dict()
for i in item_list:
    if not i in item_id_dict:
        item_id_dict[i] = j
        j += 1

In [23]:
print('sparsity: ' + str(len(rdf) * 1.0 / (len(user_list) * len(item_list))))

sparsity: 0.08710312334371163


In [24]:
# get the df of train, vali, and test set
rdf.reset_index(inplace=True, drop=True)
train_df = rdf.copy()
vali_df = rdf.copy()
test_df = rdf.copy()

train_ratio = 0.8
vali_ratio = 0.0
test_ratio = 0.2
num_all = len(rdf)
vali_idx = []
test_idx = []

test_vali_idx = []
i = 0
num_user = len(user_list)
for u in user_list:
    u_idx = train_df.index[train_df['user_id'] == u]
    idx_len = len(u_idx)
    test_len = int(idx_len * (test_ratio + vali_ratio))
    if test_len == 0:
        test_len = 1
    tmp = np.random.choice(u_idx, size=test_len, replace=False)
    test_vali_idx += tmp.tolist()
    i += 1
    if i % 5000 == 0:
        print(str(i) + '/' + str(num_user))

# tmp = (np.random.choice(range(num_all), size=(test_len+vali_len), replace=False)).tolist()
test_len = int(len(test_vali_idx) * test_ratio / (test_ratio + vali_ratio))
vali_len = int(len(test_vali_idx) - test_len)
test_idx = (np.random.choice(test_vali_idx, size=test_len, replace=False)).tolist()
vali_idx = (np.random.choice(test_vali_idx, size=vali_len, replace=False)).tolist()

test_set = set(test_idx)
vali_set = set(vali_idx)
train_set = set(range(num_all)) - test_set - vali_set
train_idx = list(train_set)
train_df.drop((test_idx + vali_idx), axis=0, inplace=True)
test_df.drop((train_idx + vali_idx), axis=0, inplace=True)
vali_df.drop((train_idx + test_idx), axis=0, inplace=True)

In [25]:
# rdf.drop(columns=['rating'], inplace=True)
# train_df.drop(columns=['rating'], inplace=True)
# test_df.drop(columns=['rating'], inplace=True)
# vali_df.drop(columns=['rating'], inplace=True)

In [26]:
train_df.shape

(26278, 5)

In [27]:
# get the matrix of train, vali and test set

train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)
vali_df.reset_index(drop=True, inplace=True)
rdf.reset_index(drop=True, inplace=True)
train = np.zeros((len(user_list), len(item_list)))
test = np.zeros((len(user_list), len(item_list)))
vali = np.zeros((len(user_list), len(item_list)))
for r in range(len(train_df)):
    train[user_id_dict[train_df.at[r, 'user_id']], item_id_dict[train_df.at[r, 'item_id']]] = 1.0
for r in range(len(test_df)):
    test[user_id_dict[test_df.at[r, 'user_id']], item_id_dict[test_df.at[r, 'item_id']]] = 1.0
for r in range(len(vali_df)):
    vali[user_id_dict[vali_df.at[r, 'user_id']], item_id_dict[vali_df.at[r, 'item_id']]] = 1.0

In [28]:
test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [29]:
# # get the user int id-> str id list, and the same for item 
# item_list = item_id_dict.keys()
# item_idd_list = list()
# for i in range(len(item_list)):
#     item_idd_list.append('')
# for item in item_id_dict:
#     item_idd_list[item_id_dict[item]] = item

# user_list = user_id_dict.keys()
# user_idd_list = list()
# for i in range(len(user_list)):
#     user_idd_list.append('')
# for user in user_id_dict:
#     user_idd_list[user_id_dict[user]] = user
    
# # get the item int id->genres list
# item_idd_genre_list = list()
# for i in range(len(item_idd_list)):
#     item_idd_genre_list.append(item_genre_dict[item_idd_list[i]])

In [30]:
# get the user int id-> str id list, and the same for item 
user_list = user_id_dict.keys()
user_idd_list = list()
for u in range(len(user_list)):
    user_idd_list.append('')
for user in user_id_dict:
    user_idd_list[user_id_dict[user]] = user

item_list = item_id_dict.keys()
item_idd_list = list()
for i in range(len(item_list)):
    item_idd_list.append('')
for item in item_id_dict:
    item_idd_list[item_id_dict[item]] = item
    
# get the user int id->types list
user_idd_type_list = list()
for u in range(len(user_idd_list)):
    user_idd_type_list.append(user_type_dict[user_idd_list[u]])

In [31]:
train_df.head()

,user_id,item_id,rating,user_freq,item_freq
0,186,302,3,89,104
1,166,346,1,20,36
2,298,474,4,126,84
3,115,265,2,92,81
4,305,451,3,221,51


In [32]:
train_df.drop('user_freq', axis=1, inplace=True)
train_df.drop('item_freq', axis=1, inplace=True)
vali_df.drop('user_freq', axis=1, inplace=True)
vali_df.drop('item_freq', axis=1, inplace=True)
test_df.drop('user_freq', axis=1, inplace=True)
test_df.drop('item_freq', axis=1, inplace=True)
rdf.drop('user_freq', axis=1, inplace=True)
rdf.drop('item_freq', axis=1, inplace=True)

In [33]:
train_df.head()

,user_id,item_id,rating
0,186,302,3
1,166,346,1
2,298,474,4
3,115,265,2
4,305,451,3


In [34]:
# get df for rdf, train, vali, test with int id for user and item
import copy
rating_df = copy.copy(rdf)
for i in range(len(rdf)):
    rating_df.at[i, 'user_id'] = user_id_dict[rating_df.at[i, 'user_id']]
    rating_df.at[i, 'item_id'] = item_id_dict[rating_df.at[i, 'item_id']]

training_df = copy.copy(train_df)
for i in range(len(training_df)):
    training_df.at[i, 'user_id'] = user_id_dict[training_df.at[i, 'user_id']]
    training_df.at[i, 'item_id'] = item_id_dict[training_df.at[i, 'item_id']]

valiing_df = copy.copy(vali_df)
for i in range(len(valiing_df)):
    valiing_df.at[i, 'user_id'] = user_id_dict[valiing_df.at[i, 'user_id']]
    valiing_df.at[i, 'item_id'] = item_id_dict[valiing_df.at[i, 'item_id']]

testing_df = copy.copy(test_df)
for i in range(len(testing_df)):
    testing_df.at[i, 'user_id'] = user_id_dict[testing_df.at[i, 'user_id']]
    testing_df.at[i, 'item_id'] = item_id_dict[testing_df.at[i, 'item_id']]

In [35]:
# generate the rating list for each key type, get the type->ratings dict
rdf.reset_index(drop=True, inplace=True)
key_type_rating = dict()
for k in key_type:
    key_type_rating[k] = 0.0
for r in range(len(rdf)):
    user = rdf.at[r, 'user_id']
    gl = user_type_dict[user]
    for k in key_type:
        if k in gl:
            key_type_rating[k] += 1.0

# get the user int id->type list
type_user_vector = dict()
for k in key_type:
    type_user_vector[k] = np.zeros((1, len(user_list)))
for u in range(len(user_idd_type_list)):
    type_list = user_idd_type_list[u]
    for t in type_list:
        if t in key_type:
            type_user_vector[t][0,u] = 1.0

In [36]:
with open("user_type_dict.pkl", "wb") as f:
    pickle.dump(user_type_dict, f, pickle.HIGHEST_PROTOCOL)
with open("type_user_vector.pkl", "wb") as f:
    pickle.dump(type_user_vector, f, pickle.HIGHEST_PROTOCOL)
with open("key_type.pkl", "wb") as f:
    pickle.dump(key_type, f, pickle.HIGHEST_PROTOCOL)
with open("user_id_dict.pkl", "wb") as f:
    pickle.dump(user_id_dict, f, pickle.HIGHEST_PROTOCOL)
with open("item_id_dict.pkl", "wb") as f:
    pickle.dump(item_id_dict, f, pickle.HIGHEST_PROTOCOL)
# with open("rdf.pkl", "wb") as f:
#     pickle.dump(rdf, f, pickle.HIGHEST_PROTOCOL)
with open("rating_df.pkl", "wb") as f:
    pickle.dump(rating_df, f, pickle.HIGHEST_PROTOCOL)
with open("training_df.pkl", "wb") as f:
    pickle.dump(training_df, f, pickle.HIGHEST_PROTOCOL)
with open("valiing_df.pkl", "wb") as f:
    pickle.dump(valiing_df, f, pickle.HIGHEST_PROTOCOL)
with open("testing_df.pkl", "wb") as f:
    pickle.dump(testing_df, f, pickle.HIGHEST_PROTOCOL)
with open("user_idd_type_list.pkl", "wb") as f:
    pickle.dump(user_idd_type_list, f, pickle.HIGHEST_PROTOCOL)
with open("item_idd_list.pkl", "wb") as f:
    pickle.dump(item_idd_list, f, pickle.HIGHEST_PROTOCOL)
with open("user_idd_list.pkl", "wb") as f:
    pickle.dump(user_idd_list, f, pickle.HIGHEST_PROTOCOL)
with open("key_type_rating.pkl", "wb") as f:
    pickle.dump(key_type_rating, f, pickle.HIGHEST_PROTOCOL)
with open("type_count.pkl", "wb") as f:
    pickle.dump(type_count, f, pickle.HIGHEST_PROTOCOL)
# with open("item_type_count.pkl", "wb") as f:
#     pickle.dump(item_type_count, f, pickle.HIGHEST_PROTOCOL)
    
with open("train.mat", "wb") as f:
    np.save(f, train)
with open("test.mat", "wb") as f:
    np.save(f, test)
with open("vali.mat", "wb") as f:
    np.save(f, vali)

In [37]:
# count the number for each user type and sort
import pickle
from operator import itemgetter
user_list = rdf['user_id'].unique()
#user_type_dict = pickle.load(open('./user_type_dict.pkl'))

with open('./user_type_dict.pkl', 'rb') as f:
    user_type_dict = pickle.load(f,encoding='latin1')

with open('./key_type.pkl', 'rb') as f:
    key_type = pickle.load(f,encoding='latin1')

#key_type = pickle.load(open('./key_type.pkl'))

type_count = dict()
for u in user_list:
    gl = user_type_dict[u]
    for g in gl:
        if g in key_type:
            if not g in type_count:
                type_count[g] = 1
            else:
                type_count[g] += 1

with open("type_count.pkl", "wb") as f:
    pickle.dump(type_count, f, pickle.HIGHEST_PROTOCOL)
                
type_count_sorted = sorted(type_count.items(), key=itemgetter(1), reverse=True)
type_count_sorted

[('educator', 95),
 ('engineer', 67),
 ('programmer', 66),
 ('executive', 32),
 ('scientist', 31),
 ('artist', 28)]

In [38]:
import numpy as np
import pickle
import copy as copy
train = np.load('./train.mat')

with open('./user_idd_type_list.pkl', 'rb') as f:
    user_idd_type_list = pickle.load(f,encoding='latin1')

user_idd_type_list = np.array(user_idd_type_list)


mask = 1.0 * (train > 0)
item_type_count = list()
for i in range(train.shape[1]):
    temp_type_count = copy.copy(type_count)
    mask_i = mask[:, i]
    gll = user_idd_type_list[mask_i == 1.0]
    for gl in gll:
        for g in gl:
            if g in key_type:
                temp_type_count[g] -= 1
    item_type_count.append(temp_type_count)
with open("item_type_count.pkl", "wb") as f:
    pickle.dump(item_type_count, f, pickle.HIGHEST_PROTOCOL)

In [39]:
len(item_type_count)

1177

In [40]:
type_count_sorted

[('educator', 95),
 ('engineer', 67),
 ('programmer', 66),
 ('executive', 32),
 ('scientist', 31),
 ('artist', 28)]

In [41]:
with open('./key_type_rating.pkl', 'rb') as f:
    key_type_rating = pickle.load(f,encoding='latin1')


#key_type_rating = pickle.load(open('./key_genre_rating.pkl'))
type_avg_like = dict()
for k in key_type:
    type_avg_like[k] = key_type_rating[k] * 1.0 / type_count[k]

In [42]:
type_avg_like_sorted = sorted(type_avg_like.items(), key=itemgetter(1), reverse=True)
type_avg_like_sorted

[('engineer', 121.23880597014926),
 ('programmer', 116.63636363636364),
 ('executive', 103.0),
 ('educator', 97.94736842105263),
 ('artist', 79.92857142857143),
 ('scientist', 65.93548387096774)]